# <center>This `.ipynb` file contains the code for the first finetuning of the `cLDM` architecture</center>

### 1. Import the required libraries

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import sys
import os
import datetime
import random

import numpy as np
from PIL import Image

from torchvision import datasets, transforms
from torchinfo import summary

sys.path.insert(0, '..')
from pfiles.unet_cond_base import UNet
from pfiles.vqvae import VQVAE
from pfiles.linear_noise_scheduler import LinearNoiseScheduler

### 2. Define a stamp to save model

In [ ]:
def timestamp():
    time_cur = datetime.datetime.now()
    stamp = time_cur.strftime('%Y%m%d')
    return stamp

In [ ]:
stmp = timestamp()
stmp

### 3. Define the device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device is:', device)

### 4. Set different hyperparameters

In [ ]:
seed = 765

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
num_timesteps = 1000
beta_start = 0.0001
beta_end = 0.02

In [ ]:
select_batch_size = 16
rgb_input = 3
z_channels = 16
n_clusters = 14 # change it to 10, 11, 12, 13, 15, or 16 for other partitions

### 5. Load the dataset

In [ ]:
dir_src = '/project/dsc-is/nono/Documents/kpc/dat0'
data_src = 'slice128_Block2_11K.npy'

print(os.path.join(dir_src, data_src))

kpc_dataset = np.load(os.path.join(dir_src, data_src))
kpc_dataset = kpc_dataset[:, 0, :, :, :]

print(kpc_dataset.shape)
N_SAMPLE, HEIGHT, WIDTH, CHANNELS = kpc_dataset.shape

In [ ]:
index_range = np.arange(N_SAMPLE)
split = np.array_split(index_range, 11)
test_dataset = split[10]
training_dataset = np.setdiff1d(index_range, test_dataset)

In [ ]:
print('Length of the training dataset:', len(training_dataset))
print('Length of the test dataset:', len(test_dataset))

### 6. Custom functions for `marginal entropy`, `conditional entropy`, and `KL divergence`

In [ ]:
class history():
    def __init__(self, keys):
        self.values = {}
        for k in keys:
            self.values[k] = []
        self.keys = keys
        
    def append(self, dict_hist):
        for k in dict_hist.keys():
            self.values[k].append(dict_hist[k])
    
    def mean(self, keys=None):
        if (keys is None):
            keys = self.keys
        m = {}
        for k in keys:
            m[k] = np.round(np.mean(self.values[k]), 4)
        return m
    
    def __getitem__(self, key):
        return (self.values[key])
    
    def __str__(self):
        get = self.mean(self.keys)
        return ('\t'.join([k + ': ' + str(get[k]) for k in self.keys]))

### 7. Custom functions for extracting batches of samples from the dataset

In [ ]:
def make_batch_list(idx, n_batch=10, batch_size=None, shuffle=True):
    if shuffle:
        np.random.shuffle(idx)
    if batch_size is not None:
        n_batch = len(idx) // batch_size
    batch_list = np.array_split(idx, n_batch)
    return batch_list

In [ ]:
transform = transforms.ToTensor()

def generate_batch(idx, kpc_dataset):
    tmp = []
    for i in idx:
        xxx = transform(kpc_dataset[i])
        tmp.append(xxx)
    xxx_batch = torch.stack(tmp, dim=0)
    return xxx_batch

### 8. Set up directory for saving models

In [ ]:
task_name = 'models_14'

if not os.path.exists(task_name):
    os.mkdir(task_name)

### 9. Instantiate `linear` scheduler

In [ ]:
scheduler = LinearNoiseScheduler(num_timesteps=num_timesteps, beta_start=beta_start, beta_end=beta_end)

### 10. Neural network for deep learning-based clustering

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.classifier = nn.Sequential()
        self.classifier.add_module('conv1', nn.Conv2d(in_channels=z_channels, out_channels=128, kernel_size=4, stride=2,
                                                      padding=1))
        self.classifier.add_module('bnor1', nn.BatchNorm2d(num_features=128, affine=True, track_running_stats=True))
        self.classifier.add_module('lrel1', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        self.classifier.add_module('conv2', nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1))
        self.classifier.add_module('bnor2', nn.BatchNorm2d(num_features=128, affine=True, track_running_stats=True))
        self.classifier.add_module('lrel2', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        self.classifier.add_module('conv3', nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1))
        self.classifier.add_module('lrel3', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        self.classifier.add_module('conv4', nn.Conv2d(in_channels=128, out_channels=n_clusters, kernel_size=4, stride=1,
                                                      padding=0))
        self.classifier.add_module('lrel4', nn.LeakyReLU(negative_slope=0.1, inplace=True))
        
    def forward(self, lat):
        out = self.classifier(lat)
        return out

### 11. Visualize `Classifier` architecture

In [ ]:
summary(Classifier(), input_size=(16, 16, 32, 32)) # batch_size, z_channels, latent_height, latent_width

### 12. Visualize `VQVAE` architecture

In [ ]:
summary(VQVAE(im_channels=rgb_input), input_size=(16, 3, 128, 128)) # batch_size, channels, height, width

### 13. Visualize `UNet` architecture

In [ ]:
summary(UNet(im_channels=z_channels, cls=n_clusters), input_size=[(16, 16, 32, 32), (16,)])
# (batch_size, z_channels, latent_height, latent_width), (batch_size)

### 14. Instantiate `UNet`, `VQVAE`, and `Classifier` architecture

In [ ]:
model = UNet(im_channels=z_channels, cls=n_clusters).to(device)
model.eval()
print('Loaded unet training checkpoint')
model.load_state_dict(torch.load(os.path.join(task_name, 'unet_training_ckpt_20250127_600_14.pth'), map_location=device,
                                 weights_only=True))

vq_vae = VQVAE(im_channels=rgb_input).to(device)
vq_vae.eval()
print('Loaded vq_vae checkpoint')
vq_vae.load_state_dict(torch.load(os.path.join('../kpc_ldm', 'vqvae_autoencoder_ckpt.pth'), map_location=device,
                                  weights_only=True))

In [ ]:
model_cl = Classifier().to(device)
model_cl.eval()
print('Loaded model_cl training checkpoint')
model_cl.load_state_dict(torch.load(os.path.join(task_name, 'classifier_training_ckpt_20250127_600_14.pth'),
                                    map_location=device, weights_only=True))

### 15. Prepare to train the `cLDM`

In [ ]:
key_loss = ['MSE']
loss_hist = history(['Epoch'] + key_loss)

# setting up additional hyperparameters
num_epochs = 130
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

### 16. Train the `cLDM `

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

for param in vq_vae.parameters():
    param.requires_grad = False

model.train()

for epoch_idx in range(num_epochs):
    batch_list = make_batch_list(training_dataset, batch_size=select_batch_size)
    
    loss_tt = history(key_loss)
    
    for idx_tmp in batch_list:
        optimizer.zero_grad()
        xxx_tmp = generate_batch(idx_tmp, kpc_dataset)
        im = xxx_tmp.to(device)

        with torch.no_grad():
            im, _ = vq_vae.encode(im)
    
        out_cl = model_cl(im)
        cond_input = torch.argmax(out_cl.reshape((-1, n_clusters)), dim=1)

        noise = torch.randn_like(im).to(device)

        t = torch.randint(low=0, high=num_timesteps, size=(im.shape[0],)).to(device)

        noisy_im = scheduler.add_noise(im, noise, t)
        noise_pred = model(noisy_im, t, cond_input=cond_input)

        mse_loss = criterion(noise_pred, noise)
        
        mse_loss.backward()
        optimizer.step()
        
        loss_tt.append({'MSE': mse_loss.item()})
    
    loss_hist.append({'Epoch': epoch_idx + 1})
    loss_hist.append(loss_tt.mean())
    
    print('Epoch:', epoch_idx + 1, '\t', str(loss_tt))

print('Done training...')

### 17. Save model after training

In [ ]:
torch.save(model.state_dict(), os.path.join(task_name, f'unet_finetuning1_ckpt_{stmp}_{num_epochs}_{n_clusters}.pth'))